# 建模强化学习问题-马尔可夫决策过程

本章涵盖
* 弦图和我们的教学方法
* PyTorch 深度学习框架
* 解决 n 臂老虎机问题
* 平衡探索与开发
* 将问题建模为马尔可夫决策过程（MDP）
* 使用神经网络解决广告选择问题

本章涵盖了所有强化学习中的一些最基本的概念，它将成为本书其余部分的基础。 但在我们开始之前，我们想先回顾一下我们将在本书中使用的一些反复出现的教学方法——最值得注意的是我们上一章提到的弦图。

## 弦图和我们的教学方法
根据我们的经验，当大多数人试图教授一些复杂的东西时，他们倾向于以与主题本身发展的相反顺序来教授它。 他们会给你一堆定义、术语、描述，也许还有定理，然后他们会说，“太好了，既然我们已经涵盖了所有的理论，让我们来解决一些实践问题。” 在我们看来，这与呈现事物的顺序完全相反。 大多数好的想法都是作为解决现实世界的问题，或者至少是想象中的问题而产生的。 问题解决者偶然发现一个潜在的解决方案，对其进行测试、改进，然后最终将其形式化并可能对其进行数学化。 术语和定义出现在问题的解决方案之后。

我们认为，当你取代那个正在思考如何解决特定问题的原创创意者时，学习是最有动力和最有效的。 只有当解决方案具体化时，它才需要形式化，这对于确定其正确性并将其忠实地传达给该领域的其他人来说确实是必要的。
 
有一种强烈的冲动要参与这种逆时间顺序的教学模式，但我们会尽最大努力抵制它，并在我们进行时发展这个话题。 本着这种精神，我们将根据需要引入新的术语、定义和数学符号。 例如，我们将使用这样的“标注”：
> 定义 神经网络是一种由多个“层”组成的机器学习模型，这些层执行矩阵向量乘法，然后应用非线性“激活”函数。 神经网络的矩阵是模型的可学习参数，通常被称为神经网络的“权重”。

每个术语您只会看到一次这些标注，但我们通常会在文本中以不同方式重复定义，以确保您真正理解并记住它。 这是一门关于强化学习的课程，而不是教科书或参考资料，因此当我们认为需要记住的事情很重要时，我们不会回避重复自己。
 
每当我们需要介绍一些数学时，我们通常会使用一个框来显示数学和同一基础概念的伪 Python 版本。 有时从代码或数学的角度思考更容易，我们认为熟悉两者是件好事。 作为一个超级简单的例子，如果我们要引入直线方程，我们会这样做：

![](https://tva1.sinaimg.cn/large/008i3skNly1gud4xljc2bj616s0c8dgz02.jpg)